In [2]:
from dataclasses import dataclass
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser
import pgeocode

%load_ext sql
%config SqlMagic.displaylimit = 0

In [3]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [3]:
API_KEY = config['GCP']['API_KEY']
radius = 1500  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [13]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None


latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")


In [ ]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


In [ ]:
places['results'][0]

In [14]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


Name: Hôtel Parc Belle-Vue
Address: 5 Av. Marie-Thérèse, 2132 Hollerich Luxembourg
Phone Number: 45 61 41 1
Website: https://www.goereshotels.com/belle-vue-en/


In [17]:
place_details['status']

'OK'

In [7]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [20]:
conn = duckdb.connect('../../data/raw/places.db')
%sql conn --alias duckdb

In [27]:
%%sql
select column_name from  information_schema.columns
where table_name = 'geonames';

Running query in 'duckdb'

column_name
adminCode1
lng
geonameId
toponymName
countryId
fcl
population
countryCode
name
fclName


In [29]:
%%sql
select * from geonames
limit 10

Running query in 'duckdb'

adminCode1,lng,geonameId,toponymName,countryId,fcl,population,countryCode,name,fclName,adminCodes1,countryName,fcodeName,adminName1,lat,fcode
LU,6.13,2960316,Luxembourg,2960313,P,76684,LU,Luxembourg,"city, village,...",{'ISO3166_2': 'LU'},Luxembourg,capital of a political entity,Luxembourg,49.61167,PPLC
ES,5.98056,2960596,Esch-sur-Alzette,2960313,P,28228,LU,Esch-sur-Alzette,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a second-order administrative division,Esch-sur-Alzette,49.49583,PPLA2
ES,6.0875,2960634,Dudelange,2960313,P,18013,LU,Dudelange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.48056,PPLA3
ES,6.01278,2960102,Schifflange,2960313,P,8155,LU,Schifflange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.50639,PPLA3
ES,5.88056,2960187,Pétange,2960313,P,7187,LU,Pétange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.55833,PPLA3
DI,6.15583,2960657,Diekirch,2960313,P,6756,LU,Diekirch,"city, village,...",{'ISO3166_2': 'DI'},Luxembourg,seat of a first-order administrative division,Diekirch,49.86778,PPLA
ES,6.10278,2960777,Bettembourg,2960313,P,7437,LU,Bettembourg,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.51861,PPLA3
LU,6.07333,2960054,Strassen,2960313,P,6006,LU,Strassen,"city, village,...",{'ISO3166_2': 'LU'},Luxembourg,seat of a third-order administrative division,Luxembourg,49.62056,PPLA3
00,6.16667,2960313,Grand Duchy of Luxembourg,2960313,A,607728,LU,Luxembourg,"country, state, region,...",None,Luxembourg,independent political entity,,49.75,PCLI
DI,6.10417,2960589,Ettelbruck,2960313,P,6364,LU,Ettelbruck,"city, village,...",{'ISO3166_2': 'DI'},Luxembourg,seat of a third-order administrative division,Diekirch,49.8475,PPLA3


In [10]:
%%sql
show tables;
# show placedetail;

Running query in 'duckdb'

name
placeDetail
places


In [30]:
# con.close()
conn.close()

In [ ]:
def get_cities_towns(api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address=Luxembourg&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        results = data['results']
        places = []
        for result in results:
            places.append(result['formatted_address'])
        return places
    else:
        print(f"Error: {data['status']}")


cities_towns = get_cities_towns(API_KEY)
print(cities_towns)


In [13]:
def get_geonames_data(country_code, username):
    base_url = "http://api.geonames.org/searchJSON"
    params = {
        "country": country_code,
        "maxRows": 1000,  # Adjust as needed
        "username": username
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if "geonames" in data:
        df = pd.DataFrame(data["geonames"])
        return df
    else:
        print("Error retrieving data:", data.get("status", {}).get("message", "Unknown error"))
        return None

In [5]:
def get_geonames_data(country_code, username, max_rows=1000):
    base_url = "http://api.geonames.org/searchJSON"
    all_data = []
    start_row = 0
    
    while True:
        params = {
            "country": country_code,
            "maxRows": max_rows,
            "username": username,
            "startRow": start_row
        }
        
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if "geonames" in data:
            geonames = data["geonames"]
            all_data.extend(geonames)
            
            if len(geonames) < max_rows:
                # We've reached the end of the data
                break
            
            start_row += max_rows
            
            # Respect rate limits
            time.sleep(1)
        else:
            print("Error retrieving data:", data.get("status", {}).get("message", "Unknown error"))
            return None
    
    df = pd.DataFrame(all_data)
    return df

In [6]:
username = config['GEONAME']['username']  # Replace with your GeoNames username
country_code = "LU"  # Luxembourg

df = get_geonames_data(country_code, username)

In [13]:
df.shape

(1259, 16)

In [14]:
df.head()

,adminCode1,lng,geonameId,toponymName,countryId,fcl,population,countryCode,name,fclName,adminCodes1,countryName,fcodeName,adminName1,lat,fcode
0,LU,6.13,2960316,Luxembourg,2960313,P,76684,LU,Luxembourg,"city, village,...",{'ISO3166_2': 'LU'},Luxembourg,capital of a political entity,Luxembourg,49.61167,PPLC
1,ES,5.98056,2960596,Esch-sur-Alzette,2960313,P,28228,LU,Esch-sur-Alzette,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a second-order administrative division,Esch-sur-Alzette,49.49583,PPLA2
2,ES,6.0875,2960634,Dudelange,2960313,P,18013,LU,Dudelange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.48056,PPLA3
3,ES,6.01278,2960102,Schifflange,2960313,P,8155,LU,Schifflange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.50639,PPLA3
4,ES,5.88056,2960187,Pétange,2960313,P,7187,LU,Pétange,"city, village,...",{'ISO3166_2': 'ES'},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.55833,PPLA3


In [16]:
%%sql
create table geonames as (select * from df);


Running query in 'duckdb'

RuntimeError: Catalog Error: Table with name "geonames" already exists!
If you need help solving this issue, send us a message: https://ploomber.io/community


In [18]:
%%sql
show geonames

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
adminCode1,VARCHAR,YES,None,None,None
lng,VARCHAR,YES,None,None,None
geonameId,BIGINT,YES,None,None,None
toponymName,VARCHAR,YES,None,None,None
countryId,VARCHAR,YES,None,None,None
fcl,VARCHAR,YES,None,None,None
population,BIGINT,YES,None,None,None
countryCode,VARCHAR,YES,None,None,None
name,VARCHAR,YES,None,None,None
fclName,VARCHAR,YES,None,None,None
